### Installing Required Libraries (Colab)

Since Colab sessions are temporary, we install required packages every time we restart:

- `textblob`: For basic NLP tasks like sentiment extraction and noun phrase parsing.
- `agno`: A lightweight framework to build and run modular AI agents.
- `duckduckgo-search`: Lets agents perform real-time web searches via DuckDuckGo.

We also download `textblob`'s corpora (e.g., taggers, stopwords) using its CLI tool to ensure all NLP features work smoothly.


In [1]:
! pip install textblob agno duckduckgo-search
! python -m textblob.download_corpora

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.7 MB/s eta 0:00:00
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


### Importing Agent Components and Tools

We now import the core building blocks from the `agno` framework:

- `Agent`: Represents an autonomous unit that can plan and act using tools.
- `OpenAIChat`: The LLM backend (like GPT-4) that powers agent reasoning.
- `DuckDuckGoTools`: Enables the agent to search the web.
- `YFinanceTools`: Allows the agent to fetch stock market data.
- `Team`: Lets us coordinate multiple agents to work together.

These modules allow us to create both single-agent and multi-agent setups.


In [2]:
!pip install agno
!pip install duckduckgo-search
!pip install groq
!pip install ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.8 MB/s eta 0:00:00


In [3]:
pip install yfinance

In [4]:
from agno.agent import Agent
from groq import Groq
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools
from agno.team import Team

### Setting the OpenAI API Key

This helper function checks if the `OPENAI_API_KEY` is already set in the environment.

If not, it securely prompts the user to enter it (without echoing it on screen), which is useful when running in environments like Colab.

This key is required for the `OpenAIChat` model to work.


**Defining a Sentiment Analysis Tool using TextBlob**

We define our first tool using the ***@tool decorator** from the agno framework.

This tool:
Accepts a string input (text).
Uses TextBlob to extract:
Polarity (range: -1 to 1): how negative or positive the sentiment is.
Subjectivity (range: 0 to 1): how opinion-based the text is.
Returns a formatted string with an interpretation: Positive, Negative, or Neutral.
This tool can now be used by agents for basic sentiment analysis tasks.

In [5]:
import os
os.environ["GROQ_API_KEY"] = ""

In [7]:
from agno.tools import tool
from textblob import TextBlob

@tool("TextBlob Sentiment Tool")
def analyze_sentiment(text: str) -> str:
    """Analyzes sentiment using TextBlob"""
    blob = TextBlob(text)
    sentiment = blob.sentiment
    polarity = sentiment.polarity
    subjectivity = sentiment.subjectivity
    interpretation = (
        "Positive" if polarity > 0 else "Negative" if polarity < 0 else "Neutral"
    )
    return (
        f"Sentiment Analysis:\n"
        f"- Polarity: {polarity:.2f}\n"
        f"- Subjectivity: {subjectivity:.2f}\n"
        f"- Interpretation: {interpretation}"
    )

### Creating a Sentiment Analysis Agent

We define a single agent named **Sentiment Analyst**.

- Uses `OpenAIChat` with GPT-4o as the reasoning model.
- Equipped with the `analyze_sentiment` tool we defined earlier.
- Role and instructions guide the agent to focus on financial news headlines.
- `show_tool_calls=True` enables visibility into tool usage during execution.
- `markdown=True` formats agent responses for cleaner output.

This agent is now ready to interpret sentiment in text inputs.


In [11]:
from agno.agent import Agent
from agno.models.groq import Groq

sentiment_agent = Agent(
    name="Sentiment Analyst",
    role="Analyze the sentiment of financial news headlines and identify overall tone.",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=[
        "You analyze sentiment from text.",
        "Accept headlines, summaries, or paragraphs.",
        "Classify sentiment into: Positive, Negative, Neutral.",
        "Provide reasoning in 2–3 lines.",
        "Never call tools. Never use function calls.",
        "Only use pure text analysis."
    ],
    markdown=True,
    tool_choice="none"    # Prevent tool calling (required for Groq)
)
'''sentiment_agent.print_response(
    "Headline: Nvidia reports record-breaking quarterly revenue from AI chips.",
    stream=True
'''
sentiment_agent.print_response(
    "Headline: Mobile provider Repustate using customer support sentiment to find customers at risk of churning",
    stream=True
)


Output()

### Creating a Web Search Agent

This agent, **Web Agent**, is designed to search the internet.

- Uses GPT-4o for reasoning.
- Equipped with `DuckDuckGoTools` for real-time web search.
- Instructions specify to always include sources in responses.
- Tool calls and markdown rendering are enabled for clarity.

Useful for retrieving up-to-date information or facts.


In [14]:
from agno.agent import Agent
from agno.models.groq import Groq

web_agent = Agent(
    name="Web Agent",
    role="Search the web manually and summarize results.",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=[
        "When the user says 'Search: ...', perform a web-search-like reasoning.",
        "Pretend you searched the internet and compile publicly known information.",
        "Always include a short list of sources (e.g., website names) at the end.",
        "Never call tools. Never use function calls.",
        "Only produce text responses.",
    ],
    markdown=True,
    tool_choice="none"    # IMPORTANT: Disables tool calling
)
#web_agent.print_response("Search: Who is the CEO of OpenAI?", stream=True)
web_agent.print_response("Search: who is Narendra Modi?", stream=True)



Output()

### Creating a Finance Data Agent

This agent, **Finance Agent**, is focused on retrieving financial market data.

- Uses GPT-4o as the reasoning model.
- Equipped with `YFinanceTools` to access:
  - Live stock prices
  - Analyst recommendations
  - Company information
- Instructions tell the agent to present results using tables.
- Tool usage and markdown formatting are enabled.

Ideal for querying real-time or historical financial insights.


In [17]:
from agno.agent import Agent
from agno.models.groq import Groq

finance_agent = Agent(
    name="Finance Agent",
    role="Provide financial analysis, stock fundamentals, earnings trends, and sector outlooks.",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=[
        "When the user says 'Finance: <query>', analyze financial information using reasoning.",
        "You do NOT have access to real-time tools.",
        "Simulate finance analysis based on public, known financial data.",
        "Use tables when presenting metrics such as revenue, EPS, valuation, etc.",
        "Always include approximate sources (e.g., 'Yahoo Finance', 'Reuters', 'Bloomberg').",
        "Never use or call tools. Never call functions.", "Provide the current share price"
    ],
    markdown=True,
    tool_choice='none'   # 🔥 IMPORTANT
)
finance_agent.print_response("Finance: NVIDIA Q4 earnings summary", stream=True)


Output()

### Creating a Multi-Agent Team

We define a coordinated agent **team** named *Apple Financial Analysis Team*.

- Uses GPT-4o as the central reasoning model.
- Team members include: `web_agent`, `finance_agent`, and `sentiment_agent`.
- Each agent contributes based on its role:
  - Web Agent gathers news
  - Finance Agent fetches stock data
  - Sentiment Agent analyzes headlines
- Instructions guide the team to compile everything into structured sections:
  News, Data Insights, and Sentiment Summary.

`mode="coordinate"` ensures agents work together to fulfill the overall task.


In [19]:
from agno.agent import Agent
from agno.models.groq import Groq

coordinator = Agent(
    name="Financial Analysis Coordinator",
    role="Coordinate WebAgent, FinanceAgent, and SentimentAgent to produce a final financial report.",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=[
        "You work with three assistants:",
        "1. WebAgent → searches and gives news summaries",
        "2. FinanceAgent → gives financial metrics and performance analysis",
        "3. SentimentAgent → analyzes sentiment of headlines",
        "",
        "Workflow:",
        "• Read the user query",
        "• Ask WebAgent using:  News: <query>",
        "• Ask FinanceAgent using: Finance: <query>",
        "• Ask SentimentAgent using: Sentiment: <query>",
        "• Combine all three responses",
        "• Produce final sections:",
        "  - Latest News",
        "  - Financial Data Insights",
        "  - Sentiment Summary",
        "  - Final Overall Assessment",
        "",
        "Never call tools or functions.",
        "Never use OpenAI tool tags.",
    ],
    markdown=True,
    tool_choice="none"
)
query = "Analyze the financial outlook and market sentiment for AI semiconductor companies"

# 1. Get news
web_resp = web_agent.run(f"News: {query}")

# 2. Get financial data
finance_resp = finance_agent.run(f"Finance: {query}")

# 3. Get sentiment
sentiment_resp = sentiment_agent.run(f"Sentiment: {query}")
query = "What's the market outlook and financial performance of broadcom?"

news_result = web_agent.run(f"News: {query}", stream=False)
finance_result = finance_agent.run(f"Finance: {query}", stream=False)
sentiment_result = sentiment_agent.run(f"Sentiment: {query}", stream=False)

# 4. Combine using coordinator
coordinator.print_response(
    f"Combine the following:\n\n"
    f"=== NEWS ===\n{news_result}\n\n"
    f"=== FINANCIAL DATA ===\n{finance_result}\n\n"
    f"=== SENTIMENT ===\n{sentiment_result}\n\n"
    f"Write a structured 4-section financial intelligence report.",
    stream=True
)



Output()

### Running a Custom Prompt with the Agent Team

We ask the team to:

> "Please give top 5 news of Microsoft and analyze the sentiment"

This prompt focuses more on news analysis:

- Web Agent retrieves the top 5 Microsoft-related headlines.
- Sentiment Agent evaluates the tone of each headline.
- Finance Agent may stay idle if not needed for this task.

The team still works in coordination, responding with structured markdown output.


In [20]:
query = "Microsoft latest financial and technology news"

# Fetch top 5 Microsoft news
news_result = web_agent.run(
    f"News: Give the top 5 latest news about Microsoft with source URLs.",
    stream=False
)

# Sentiment analysis of the news
sentiment_result = sentiment_agent.run(
    f"Sentiment: Analyze sentiment of these headlines and give sentiment score between -1 and +1.\n\n{news_result}",
    stream=False
)

# Stock price impact
finance_result = finance_agent.run(
    "Finance: Fetch Microsoft (MSFT) latest stock price trend for last 7 days and analyze if sentiment correlates.",
    stream=False
)



coordinator.print_response(
    f"Combine the following into a structured report:\n\n"

    f"=== TOP 5 MICROSOFT NEWS ===\n{news_result}\n\n"

    f"=== SENTIMENT ANALYSIS ===\n{sentiment_result}\n\n"

    f"=== STOCK PRICE ANALYSIS ===\n{finance_result}\n\n"

    f"Create a final answer explaining:\n"
    f"- The top 5 news\n"
    f"- Sentiment scores\n"
    f"- Whether sentiment impacted Microsoft’s stock price\n"
    f"- Provide a conclusion for investors.",

    stream=True
)


Output()

### Summary: Building Modular AI Agents with agno

In this demo, we explored how to build modular AI agents using the `agno` framework:

- Created individual agents for **sentiment analysis**, **web search**, and **financial data retrieval**.
- Combined them into a **coordinated team** that can collaboratively handle complex prompts.
- Used natural language instructions to direct each agent’s role and behavior.
- You can also try other web search tools like 'from agno.tools.serpapi import SerpApiTools' if you face rate limits issue with DuckDuckGoTools

This approach shows how agentic AI systems can be composed, reused, and scaled for real-world tasks like financial analysis, research, and beyond.
